##### Fernando Amaral
##### MinMaxScaler 

In [ ]:
# Transforms data to a threshold. Usually between zero and 1

In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("standardscaler").getOrCreate()

In [2]:
from pyspark.ml.feature import MinMaxScaler

In [3]:
carros = spark.read.csv("Carros.csv", header=True, inferSchema=True, sep=";")
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [4]:
# create a new column with features vectorized
from pyspark.ml.feature import VectorAssembler
vecassembler = VectorAssembler(inputCols=["Consumo","Cilindros","Cilindradas"], outputCol="vetor")
carros_vetor = vecassembler.transform(carros)
carros_vetor.select("Consumo","Cilindros","Cilindradas","vetor").show()

+-------+---------+-----------+------------------+
|Consumo|Cilindros|Cilindradas|             vetor|
+-------+---------+-----------+------------------+
|     21|        6|        160|  [21.0,6.0,160.0]|
|     21|        6|        160|  [21.0,6.0,160.0]|
|    228|        4|        108| [228.0,4.0,108.0]|
|    214|        6|        258| [214.0,6.0,258.0]|
|    187|        8|        360| [187.0,8.0,360.0]|
|    181|        6|        225| [181.0,6.0,225.0]|
|    143|        8|        360| [143.0,8.0,360.0]|
|    244|        4|       1467|[244.0,4.0,1467.0]|
|    228|        4|       1408|[228.0,4.0,1408.0]|
|    192|        6|       1676|[192.0,6.0,1676.0]|
|    178|        6|       1676|[178.0,6.0,1676.0]|
|    164|        8|       2758|[164.0,8.0,2758.0]|
|    173|        8|       2758|[173.0,8.0,2758.0]|
|    152|        8|       2758|[152.0,8.0,2758.0]|
|    104|        8|        472| [104.0,8.0,472.0]|
|    104|        8|        460| [104.0,8.0,460.0]|
|    147|        8|        440|

In [7]:
# min and max set the bounds
escala = MinMaxScaler(inputCol="vetor", outputCol="minmaxscale", min=0, max=1)
modelo = escala.fit(carros_vetor)
carrosminmax = modelo.transform(carros_vetor)
carrosminmax.select("vetor","minmaxscale").show(truncate=False)

+------------------+---------------------------------------------+
|vetor             |minmaxscale                                  |
+------------------+---------------------------------------------+
|[21.0,6.0,160.0]  |[0.05555555555555555,1.5,0.09070548712206047]|
|[21.0,6.0,160.0]  |[0.05555555555555555,1.5,0.09070548712206047]|
|[228.0,4.0,108.0] |[1.972222222222222,0.0,0.032474804031354984] |
|[214.0,6.0,258.0] |[1.8425925925925926,1.5,0.20044792833146696] |
|[187.0,8.0,360.0] |[1.5925925925925926,3.0,0.3146696528555431]  |
|[181.0,6.0,225.0] |[1.537037037037037,1.5,0.16349384098544234]  |
|[143.0,8.0,360.0] |[1.1851851851851851,3.0,0.3146696528555431]  |
|[244.0,4.0,1467.0]|[2.1203703703703702,0.0,1.5543113101903696]  |
|[228.0,4.0,1408.0]|[1.972222222222222,0.0,1.4882418812989922]   |
|[192.0,6.0,1676.0]|[1.6388888888888888,1.5,1.788353863381859]   |
|[178.0,6.0,1676.0]|[1.509259259259259,1.5,1.788353863381859]    |
|[164.0,8.0,2758.0]|[1.3796296296296295,3.0,3.0]              